In [1]:
import fitbit
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import time
import glob

In [2]:
CLIENT_ID =  "********"
CLIENT_SECRET  = "********"
ACCESS_TOKEN =  "********"
REFRESH_TOKEN =  "********"

In [3]:
# データ取得の期間
target_date_list = pd.date_range(pd.to_datetime("2016-04-01"), pd.to_datetime("2017-03-31"))
target_str = [target.strftime('%Y-%m-%d') for target in target_date_list]

target_str[:5]

['2016-04-01', '2016-04-02', '2016-04-03', '2016-04-04', '2016-04-05']

In [4]:
# fitbitのclientを作成
authd_client = fitbit.Fitbit(CLIENT_ID, CLIENT_SECRET
                             ,access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN)

In [14]:
# 心拍数を取得（1分単位）
# 'activities/heart'
def get_heart_data(target_date, time_interval):
    data_sec = authd_client.intraday_time_series('activities/heart', target_date, detail_level=time_interval) #'1sec', '1min', or '15min'
    if data_sec["activities-heart-intraday"]["dataset"]:
        heart_sec = json_normalize(data_sec["activities-heart-intraday"]["dataset"])
        heart_sec["date"] = target_date
        heart_sec.head()
        heart_sec.to_csv('data/{0}_heartbeat.csv'.format(target_date), index=False)
        time.sleep(30)
        return heart_sec

In [15]:
# データの取得
df_list = [get_heart_data(t_target, "1min") for t_target in target_str]

In [9]:
# 取得したデータをすべて読み込み
hb_path = glob.glob("./data/raw/*.csv")
hb_df = pd.concat([pd.read_csv(path) for path in hb_path])

In [12]:
# データの確認
hb_df.head()

,datetime,value
0,2016-04-01 02:19:00,125
1,2016-04-01 02:20:00,127
2,2016-04-01 02:21:00,120
3,2016-04-01 02:22:00,113
4,2016-04-01 02:23:00,113


In [11]:
# date と　timeを結合する
hb_df["datetime"] = pd.to_datetime(hb_df.date + " " + hb_df.time)
hb_df = hb_df[["datetime", "value"]]
hb_df.reindex()
hb_df.head()

,datetime,value
0,2016-04-01 02:19:00,125
1,2016-04-01 02:20:00,127
2,2016-04-01 02:21:00,120
3,2016-04-01 02:22:00,113
4,2016-04-01 02:23:00,113


In [17]:
datetime_list = list(hb_df.datetime)
print(datetime_list[0], datetime_list[-1])

2016-04-01 02:19:00 2017-03-31 23:59:00


In [18]:
expand_list = pd.date_range(datetime_list[0], datetime_list[-1], freq="min")

In [24]:
# 日付の全パターンを作成
expand_df = pd.DataFrame(expand_list, columns=["datetime"])
expand_df.head()

,datetime
0,2016-04-01 02:19:00
1,2016-04-01 02:20:00
2,2016-04-01 02:21:00
3,2016-04-01 02:22:00
4,2016-04-01 02:23:00


In [34]:
# Nanを補完
all_heartbeat = pd.merge(expand_df, hb_df, on=['datetime'], how="left")
all_heartbeat.interpolate(inplace=True)
all_heartbeat.head()

,datetime,value
0,2016-04-01 02:19:00,125.0
1,2016-04-01 02:20:00,127.0
2,2016-04-01 02:21:00,120.0
3,2016-04-01 02:22:00,113.0
4,2016-04-01 02:23:00,113.0


In [35]:
all_heartbeat.to_csv('./data/20160401_20170331_heartbeat.csv', index=None)